In [56]:
# Import the necessary functions
import logging
# Set the logging level to WARNING to suppress INFO messages
logging.basicConfig(level=logging.WARNING)
import transformers

#logging.set_verbosity_error()
# Disable specific warnings
transformers.logging.set_verbosity_error()
from inference_utils import load_model_and_tokenizer, generate_predictions, setup_device
import json #, jsonlines

from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer
import torch

from tqdm import tqdm
import re

In [57]:
# Generate predictions
# Define the parameters for generation
max_length = 10  # Adjust the max_length as needed
num_return_sequences = 1  # Adjust the number of return sequence

#predictions = generate_predictions(model, tokenizer, input_texts, device, max_length=max_length, num_return_sequences=num_return_sequences)
def predict(model, tokenizer, input_texts, device, max_length=max_length, num_return_sequences=num_return_sequences):
    predictions = generate_predictions(model, tokenizer, input_texts, device, max_length=max_length, num_return_sequences=num_return_sequences)
    # Print the predictions
    for i, prediction in enumerate(predictions):
        print(f"Input: {input_texts[i]}")
        print(f"Prediction: {prediction}")

In [58]:
# To chose the "B" that compare 2 times both in ID and OOD**
def chose_B(range_):
    list_ID=[]
    list_OOD=[]
    list_inferred_ID=[]
    list_inferred_OOD=[]

    count=0
    for i in range(range_):
        target = f'<e_{i}></a>'
        t_print=f'<e_{i}>'
        filtered_texts = [entry['target_text'] for entry in d['id_atomic'] if entry['target_text'].endswith(target)]

        filtered_texts_OOD = [entry['target_text'] for entry in d['ood_atomic'] if entry['target_text'].endswith(target)]
        if count <=2000:
            if len(filtered_texts) >= 1 and len(filtered_texts_OOD) >= 1:

                filtered_texts_2_hop = [entry['target_text'] for entry in d['id_atomic'] if entry['target_text'].startswith(t_print)]

                filtered_texts_OOD_2_hop = [entry['target_text'] for entry in d['ood_atomic'] if entry['target_text'].startswith(t_print)]

                # Extract the part before <e_57></a> from the first element
                #atomic_part_1_id = filtered_texts[0].split(target)[0]
                #print("atomic_id", atomic_part_1_id)
                #atomic_part_1_ood = str(filtered_texts_OOD[0].split(target)[0])

                #inferred_OOD = [entry['target_text'] for entry in d['test_inferred_ood']
                #                    if entry['type'] == 'test_inferred_ood' and entry['input_text'].startswith(atomic_part_1_ood)]

                #inferred_ID = [entry['target_text'] for entry in d['test_inferred_iid']
                #                    if entry['type'] == 'test_inferred_iid' and entry['input_text'].startswith(atomic_part_1_id)]

                if len(filtered_texts_2_hop) >= 1 and len(filtered_texts_OOD_2_hop) >= 1:
                    

                    list_ID.append(filtered_texts)
                    list_OOD.append(filtered_texts_OOD)
                    list_inferred_ID.append(filtered_texts_2_hop)
                    list_inferred_OOD.append(filtered_texts_OOD_2_hop,)

                    if count <=3:
                        print("-------Target :", t_print,  "     ----------------------------------------------------------\n")
                        print("ID",filtered_texts   ,"\n")
                        print("OOD",filtered_texts_OOD   ,"\n \n")

                        #print("Inferred _ ID", inferred_ID)
                        #print("Inferred _ OOD", inferred_OOD)
                        # for the second hop

                        count+=1
                        print("2- hop  ID", filtered_texts_2_hop)
                        print("2nd hop OOD", filtered_texts_OOD_2_hop, "\n \n \n")
                        
    return list_ID, list_OOD, list_inferred_ID, list_inferred_OOD


In [59]:
def return_rank(hd, word_embedding_, token, metric='dot', token_list=None):
    """
hd: Hidden states or output from a neural network.
word_embedding_: Embedding matrix for words. (matrix that convert words to their embedding representation)
token: The specific token (word) for which we want to find the rank.
metric: The similarity metric to use ('dot' for dot product, 'cos' for cosine similarity).
token_list: Optional list of tokens to consider for ranking."""

    if metric == 'dot':
        word_embedding = word_embedding_
    elif metric == 'cos':
        word_embedding = F.normalize(word_embedding_, p=2, dim=1)
    else:
        assert False
    #Compute the similarity scores (logits) between the hidden states (hd) and the word embeddings using matrix multiplication.
    logits_ = torch.matmul(hd, word_embedding.T)  # a vlaue higher if he similarity with the analyzed "word" is higehr

    rank = [] 
    for j in range(len(logits_)):
        log = logits_[j].cpu().numpy()
        if token_list is None:
            temp = [[i, log[i]] for i in range(len(log))]
        else:
            temp = [[i, log[i]] for i in token_list]
        temp.sort(key=lambda var: var[1], reverse=True)
        rank.append([var[0] for var in temp].index(token))
    return rank


In [60]:
import torch
import torch.nn.functional as F
from torch.nn import LayerNorm

def return_rank_norm_layer(hd, word_embedding_, token, metric='dot', token_list=None, layer_norm=None):
    """
    Returns the rank of a given token based on similarity between hidden states and word embeddings.

    Args:
        hd (torch.Tensor): Hidden states of shape (batch_size, hidden_dim).
        word_embedding_ (torch.Tensor): Word embedding matrix of shape (vocab_size, hidden_dim).
        token (int): Target token index whose rank is to be found.
        metric (str): Similarity metric ('dot' or 'cos'). Defaults to 'dot'.
        token_list (list, optional): List of token indices to consider. Defaults to None.
        layer_norm (LayerNorm, optional): LayerNorm instance for final layer normalization. 
                                          If None, no normalization is applied. Defaults to None.

    Returns:
        list: Ranks of the target token for each instance in the batch.
    """

    # Apply final layer normalization if provided
    if layer_norm is not None:
        hd = layer_norm(hd)

    # Normalize word embeddings if metric is cosine similarity
    if metric == 'cos':
        word_embedding = F.normalize(word_embedding_, p=2, dim=1)
    elif metric == 'dot':
        word_embedding = word_embedding_
    else:
        raise ValueError("Invalid metric. Use 'dot' or 'cos'.")

    # Compute logits by projecting hidden states onto the embedding matrix
    logits_ = torch.matmul(hd, word_embedding.T)

    # Calculate rank of the target token
    rank = []
    for log in logits_:
        log_np = log.cpu().numpy()
        candidates = [[i, log_np[i]] for i in (token_list if token_list is not None else range(len(log_np)))]
        candidates.sort(key=lambda var: var[1], reverse=True)
        rank.append([var[0] for var in candidates].index(token))

    return rank


#dataset = "/home/s220331/GROK/Thesis/data/composition_SMALL.200.20.12.6"
dataset="/home/s220331/GROK/Thesis/data/composition.2000.200.12.6"
model_path = "/dtu-compute/s220331/composition/outputs_BIG_new/checkpoint-1500000/"
#"/dtu-compute/s220331/composition/outputs_BIG_new/checkpoint-1500000/" 

target_layer = 8


# model set up
#device = torch.device('cuda:5')
device = setup_device()
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
word_embedding = model.lm_head.weight.data
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id

In [61]:
#test case to remove
#dataset = "/home/s220331/GROK/Thesis/data/composition_SMALL.200.20.12.6"
#model_path = "/scratch/davide/model_paper/outputs_SMALL/checkpoint-4000/" 

dataset="/home/s220331/GROK/Thesis/data/composition.2000.200.12.6"  
#model_path = "/dtu-compute/s220331/composition/outputs_BIG_extream_training/checkpoint-1200000/"
#model_path = "/dtu-compute/s220331/composition/outputs_BIG_extream_training/checkpoint-2250000/" # "/dtu-compute/s220331/composition/outputs_BIG_new/checkpoint-1500000/"
#model_path = "/dtu-compute/s220331/composition/outputs_BIG_extream_training/checkpoint-50000/" # "/dtu-compute/s220331/composition/outputs_BIG_new/checkpoint-1500000/"
#model_path = "/dtu-compute/s220331/composition/outputs_BIG_new/checkpoint-700000/"
#model_path = "/dtu-compute/s220331/composition/outputs_BIG_extream_training/checkpoint-700000/"
model_path = "/dtu-compute/s220331/composition/outputs_BIG_extream_training/checkpoint-400000/"
#model_path = "/dtu-compute/s220331/composition/outputs_BIG_extream_training/checkpoint-800000/"
#model_path = "/scratch/davide/model_paper/outputs_SMALL/checkpoint-4000/" 
target_layer = 8
# model set up
#device = torch.device('cuda:5')
device = setup_device()
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
word_embedding = model.lm_head.weight.data
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id

In [62]:


# set data

all_atomic = set()     # (h,r,t)
atomic_dict = dict()   # (h,r) -> t
with open(dataset+"/train.json") as f:  # from the correct data or data_MIO !!!
    train_items = json.load(f)
for item in tqdm(train_items):
    temp = item['target_text'].strip("><").split("><")
    if len(temp) != 4:
        continue
    h,r,t = temp[:3]
    atomic_dict[(h,r)] = t
    all_atomic.add((h,r,t))

id_atomic = set()
for item in tqdm(train_items):
    temp = item['target_text'].strip("><").split("><")
    if len(temp) == 4:
        continue
    h, r1, r2, t = temp[:4]
    b = atomic_dict[(h, r1)]
    assert atomic_dict[(b, r2)] == t
    id_atomic.add((h,r1,b))
    id_atomic.add((b,r2,t))

ood_atomic = all_atomic - id_atomic
print("# id_atomic, # ood_atomic:", len(id_atomic), len(ood_atomic))

# smart way to save all the train
with open(dataset+"/test.json") as f:
    pred_data = json.load(f)
d = dict()
for item in pred_data:
    t = item['type']
    if t not in d:
        d[t] = []
    d[t].append(item)

100%|██████████| 518800/518800 [00:01<00:00, 400075.84it/s]

# id_atomic, # ood_atomic: 38000 2000


**To chose the "B" that compare 2 times both in ID and OOD**

In [63]:
#a= chose_B(199)
#a[1][0]



In [64]:
query_change_OOD ="<e_147><r_4><r_1>" #"<e_181><r_6><r_5>"# "<e_147><r_4><r_1>"# "<e_14><r_12><r_13>"       # 2 hop ['<e_11><r_1><e_141></a>'] 

def run_query(query_change_OOD, ood=False):
    # inser here the 2-hop query (like ood) from whihc extract the first hop-- "b" activations
    # usato "<e_147><r_4><r_1>"
    h_1_ood,n_r_1_ood,r_1_ood = query_change_OOD.strip("><").split("><") #"e_140","r_19","r_11"             #query.strip("><").split("><")
    b_1_ood = atomic_dict[(h_1_ood ,n_r_1_ood)]
    t_1_ood = atomic_dict[(b_1_ood, r_1_ood)]
    list_b_t=[]

    print("b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa ",b_1_ood)
    print("tail:",t_1_ood)


    # Tokenizing the Query:
    decoder_temp_1_OOD = tokenizer([query_change_OOD], return_tensors="pt", padding=True)
    decoder_input_ids_1_OOD, decoder_attention_mask_1_OOD = decoder_temp_1_OOD["input_ids"], decoder_temp_1_OOD["attention_mask"]
    decoder_input_ids_1_OOD, decoder_attention_mask_1_OOD = decoder_input_ids_1_OOD.to(device), decoder_attention_mask_1_OOD.to(device)


    with torch.no_grad():
        # here the same model but with different input !! 
        outputs_1_OOD = model(
            input_ids=decoder_input_ids_1_OOD,
            attention_mask=decoder_attention_mask_1_OOD,
            output_hidden_states=True
        )
    # hidden state of the 1_id (which will be inserted in the normal run)
    all_hidden_states_1_OOD = outputs_1_OOD['hidden_states']

    #################### just to quick check everything is right

    res_dict_b_1_OOD = dict()
    target_layer_b = 5
    target_layer_t = 8
    # ("<"+ b +">")  is just for reconstructing ("<b>") from "b"
    rank_before_1_OOD = return_rank_norm_layer(all_hidden_states_1_OOD[target_layer_b][0, :, :], word_embedding, tokenizer("<"+ b_1_ood +">")['input_ids'][0])[-2]
    res_dict_b_1_OOD['rank_before, layer 5 b'] = rank_before_1_OOD

    # laste layer search for t
    rank_before_1_OOD_t = return_rank_norm_layer(all_hidden_states_1_OOD[target_layer_t][0, :, :], word_embedding, tokenizer("<"+ t_1_ood +">")['input_ids'][0])[-1]
    res_dict_b_1_OOD['rank_before, layer 8 search t'] = rank_before_1_OOD_t
    list_b_t.append((rank_before_1_OOD, rank_before_1_OOD_t))
    if ood:
        print("OOD")
        print(" B 1_OOD",res_dict_b_1_OOD)
        print("as expected in the case of OOD the B is corrctly found at layer 5 but not T at layer 8")
    else:
        print("ID")
        print(" B 1_OOD",res_dict_b_1_OOD)
        print("as expected in the case of ID the B is corrctly found at layer 5 and T at layer 8")

    return all_hidden_states_1_OOD , b_1_ood ,t_1_ood ,list_b_t



In [65]:
res_dict_OOD_intervention= dict()
def replace_hidden_state(hidden_to_insert,b, all_hidden_states_to_change,t):
    list_intervention_results = []
    target_layer_intervention = 5
    target_layer_final = 8
    # juest rename for more standardized structure
    all_hidden_states_ctft_1 = hidden_to_insert

    # we are analysing  with replacing only specific hidden state once a time
    for layer_to_intervene in range(target_layer_intervention , target_layer_intervention +1):
        print(layer_to_intervene)
        hidden_states_1 = all_hidden_states_to_change[layer_to_intervene].clone()    # cloniamo gli hidden del normal run
        hidden_states_ctft_1 = all_hidden_states_ctft_1[layer_to_intervene]  # copiamo quelli del run che inseriremo
    #r_2  r
        #r_2_ood = all_hidden_states_1_OOD_test[layer_to_intervene]

        # intervene
        hidden_states_1[0, 0, :] = hidden_states_ctft_1[0, 0, :]   #insereiamo gli hidden state che vogliamo
        hidden_states_1[0, 1, :] = hidden_states_ctft_1[0, 1, :]
        #hidden_states_1[0, 2, :] = hidden_states_ctft_1[0, 2, :]  # prova ad inserire r_2 di ood
        #hidden_states_1[0, 2, :]= r_11_id
        #hidden_states_1[0, 2, :]= r_2_ood[0, 2, :] # this is a r(second hop) coming from another ood (with not even a;r_1 the same. This prove that r_2 does not 
        # store calculation in the first 5 layers according to paper results.)
        

        rank_middle_1 = return_rank_norm_layer(hidden_states_1[0, :, :], word_embedding, tokenizer("<"+b+">")['input_ids'][0])[-2]
        res_dict_OOD_intervention["insertion at layer"+str(layer_to_intervene), "B"] = rank_middle_1
        #                
        with torch.no_grad():
            for i in range(layer_to_intervene, target_layer_final):
                f_layer_1 = model.transformer.h[i]  # current layer
                print("layer aggiornato:",i)
                # attention mechanism 
                residual_1 = hidden_states_1       #store the hidden state in residual
                # Apply layer normalization to hidden_states.
                hidden_states_1 = f_layer_1.ln_1(hidden_states_1)
                attn_output_1 = f_layer_1.attn(hidden_states_1)[0] 
                hidden_states_1 = attn_output_1 + residual_1
                # mlp  = Feed-Forward Network:
                residual_1 = hidden_states_1
                hidden_states_1 = f_layer_1.ln_2(hidden_states_1)
                feed_forward_hidden_states_1 = f_layer_1.mlp.c_proj(f_layer_1.mlp.act(f_layer_1.mlp.c_fc(hidden_states_1)))
                hidden_states_1 = residual_1 + feed_forward_hidden_states_1
            # final ln
            hidden_states_1 = model.transformer.ln_f(hidden_states_1)
        # print("--------")
        rank_after_1 = return_rank(hidden_states_1[0, :, :], word_embedding, tokenizer("<"+t+">")['input_ids'][0])[-1]
        res_dict_OOD_intervention["Out-tput layer"+str(target_layer_final), "t"] = rank_after_1
        list_intervention_results.append(rank_middle_1)
        list_intervention_results.append(rank_after_1)
        print("intervention results",res_dict_OOD_intervention)
    print( list_intervention_results)
    return list_intervention_results


## **Intervention with OOD insertion**

What: Insert in the normal run (ID run) the 5 layer hydden layers coming from a OOD run representing the same Bridge entity and r_2 relation.
It has be seen that the model still predict B (as expected as also the OOD predict the B) but also the t. In other words, does not seem that the model store and use information about how he gets to the Bridge. Given one Bridge he is able to secondo hop if the second hop is in ID.

In [66]:
atomic_to_chose = chose_B(200)

-------Target : <e_1>      ----------------------------------------------------------

ID ['<e_1678><r_82><e_1></a>'] 

OOD ['<e_1182><r_46><e_1></a>', '<e_1270><r_14><e_1></a>'] 
 

2- hop  ID ['<e_1><r_157><e_1608></a>']
2nd hop OOD ['<e_1><r_158><e_530></a>', '<e_1><r_121><e_764></a>', '<e_1><r_199><e_1914></a>', '<e_1><r_189><e_736></a>'] 
 
 

-------Target : <e_4>      ----------------------------------------------------------

ID ['<e_453><r_87><e_4></a>'] 

OOD ['<e_1905><r_115><e_4></a>', '<e_1928><r_174><e_4></a>', '<e_1255><r_153><e_4></a>'] 
 

2- hop  ID ['<e_4><r_11><e_363></a>', '<e_4><r_173><e_1710></a>']
2nd hop OOD ['<e_4><r_23><e_440></a>'] 
 
 

-------Target : <e_17>      ----------------------------------------------------------

ID ['<e_437><r_177><e_17></a>'] 

OOD ['<e_1743><r_113><e_17></a>', '<e_641><r_129><e_17></a>'] 
 

2- hop  ID ['<e_17><r_193><e_1489></a>']
2nd hop OOD ['<e_17><r_93><e_1659></a>'] 
 
 

-------Target : <e_18>      ----------------------

In [67]:


# take 1 from 1_id and 1 from inferred to construct the query
ID_ID_list = []
OOD_OOD_list = []
OOD_intervention_in_ID= []
for i in range(0,50):
    _1_id = atomic_to_chose[0][i]  # list with all the
    _1_ood = atomic_to_chose[1][i]
    _1_id_inferred = atomic_to_chose[2][i]
    _1_ood_inferred = atomic_to_chose[3][i]

    text_1 = _1_id[0]
    match = re.match(r'^(<e_\d+><r_\d+>)', text_1)
    if match:
        result_1_id = match.group(1)
    text_2= _1_id_inferred[0]
    match = re.match(r'^<e_\d+>(<r_\d+>)', text_2)
    if match:
        result2_id = match.group(1)

    text_3 = _1_ood[0]
    match = re.match(r'^(<e_\d+><r_\d+>)', text_3)
    if match:
        result_1_ood = match.group(1)
    text_4= _1_ood_inferred[0]
    match = re.match(r'^<e_\d+>(<r_\d+>)', text_4)
    if match:
        result2_ood = match.group(1)

    id_id = result_1_id + result2_id
    ood_ood = result_1_ood + result2_ood
    print("ID_ID---------------------------------------------------------------------------", id_id)   
    print("OOD_OOD------------------------------------------------------------------------------------", ood_ood)
    normal_run_ID_ID =run_query(id_id, ood=False)
    B_OOD_insert_activations =run_query(ood_ood, ood=True) 
    print("-------------------------------------------------------------------------------------------------------------------b",B_OOD_insert_activations[1])
    
    intervention=  replace_hidden_state(B_OOD_insert_activations[0],B_OOD_insert_activations[1],normal_run_ID_ID[0], normal_run_ID_ID[2])  # hidden to insert, b, (all hidden states) normal run, t
    
    ID_ID_list.append(normal_run_ID_ID[3])
    OOD_OOD_list.append(B_OOD_insert_activations[3])
    OOD_intervention_in_ID.append(intervention)  



ID_ID--------------------------------------------------------------------------- <e_1678><r_82><r_157>
OOD_OOD------------------------------------------------------------------------------------ <e_1182><r_46><r_158>
b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa  e_1
tail: e_1608
Alpha scaling found. Using alpha = 1


ID
 B 1_OOD {'rank_before, layer 5 b': 0, 'rank_before, layer 8 search t': 0}
as expected in the case of ID the B is corrctly found at layer 5 and T at layer 8
b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa  e_1
tail: e_530
Alpha scaling found. Using alpha = 1
OOD
 B 1_OOD {'rank_before, layer 5 b': 0, 'rank_before, layer 8 search t': 725}
as expected in the case of OOD the B is corrctly found at layer 5 but not T at layer 8
-------------------------------------------------------------------------------------------------------------------b e_1
5
layer aggiornato: 5
layer aggiornato: 6
layer aggiornato: 7
intervention results {('insertion at layer5', 'B'): 0, ('Out-tput layer8', 't'): 117}
[0, 117]
ID_ID--------------------------------------------------------------------------- <e_453><r_87><r_11>
OOD_OOD------------------------------------------------------------------------------------ <e_1905><r_115><r_23>
b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

In [68]:
len(atomic_to_chose[1])


52

In [69]:
len(OOD_OOD_list)

50

In [70]:
OOD_intervention_in_ID

[[0, 117],
 [0, 0],
 [0, 427],
 [0, 51],
 [1, 112],
 [0, 0],
 [0, 670],
 [1, 55],
 [0, 360],
 [0, 233],
 [0, 1],
 [0, 1],
 [3, 291],
 [0, 0],
 [0, 3],
 [0, 237],
 [0, 473],
 [0, 67],
 [0, 0],
 [0, 180],
 [0, 0],
 [1, 28],
 [0, 300],
 [0, 44],
 [0, 0],
 [1, 236],
 [4, 110],
 [0, 20],
 [1, 136],
 [0, 18],
 [0, 50],
 [0, 35],
 [1, 1272],
 [0, 41],
 [0, 181],
 [0, 918],
 [0, 583],
 [0, 369],
 [0, 27],
 [0, 439],
 [1, 6],
 [6, 197],
 [0, 154],
 [0, 0],
 [0, 3],
 [0, 1],
 [0, 9],
 [0, 0],
 [0, 0],
 [0, 0]]

In [71]:
ID_ID_list

[[(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)],
 [(0, 0)]]

In [72]:
import pandas as pd

# Your data: list of lists with two values each.
data1 = OOD_intervention_in_ID

# Create a DataFrame with appropriate column names
df1 = pd.DataFrame(data1, columns=["first", "second"])

# DataFrame 1: Rows where both 'first' and 'second' are nonzero
df_both_nonzero1 = df1[(df1["first"] == 0) & (df1["second"] == 0)].reset_index(drop=True)

# DataFrame 2: Rows where 'first' is nonzero and 'second' is zero
df_first_nonzero_second_zero1 = df1[(df1["first"] != 0) & (df1["second"] == 0)].reset_index(drop=True)

# Display the results
print("DataFrame with both nonzero values:")
print(df_both_nonzero1)

print("\nDataFrame with first nonzero and second zero:")
print(df_first_nonzero_second_zero1)


DataFrame with both nonzero values:
   first  second
0      0       0
1      0       0
2      0       0
3      0       0
4      0       0
5      0       0
6      0       0
7      0       0
8      0       0
9      0       0

DataFrame with first nonzero and second zero:
Empty DataFrame
Columns: [first, second]
Index: []


In [73]:
stop

NameError: name 'stop' is not defined

In [ ]:
stop

In [ ]:
import pandas as pd
#case_2MIL= OOD_intervention_in_ID.copy()
# Your data: list of lists with two values each.
data = case_2MIL

# Create a DataFrame with appropriate column names
df = pd.DataFrame(data, columns=["first", "second"])

# DataFrame 1: Rows where both 'first' and 'second' are nonzero
df_both_nonzero = df[(df["first"] == 0) & (df["second"] == 0)].reset_index(drop=True)

# DataFrame 2: Rows where 'first' is nonzero and 'second' is zero
df_first_nonzero_second_zero = df[(df["first"] != 0) & (df["second"] == 0)].reset_index(drop=True)

# Display the results
print("DataFrame with both nonzero values:")
print(df_both_nonzero)

print("\nDataFrame with first nonzero and second zero:")
print(df_first_nonzero_second_zero)


DataFrame with both nonzero values:
    first  second
0       0       0
1       0       0
2       0       0
3       0       0
4       0       0
5       0       0
6       0       0
7       0       0
8       0       0
9       0       0
10      0       0
11      0       0
12      0       0
13      0       0
14      0       0

DataFrame with first nonzero and second zero:
   first  second
0      1       0
1      4       0
2      1       0
3     27       0
4      1       0
5     17       0


In [ ]:
stop

NameError: name 'stop' is not defined

In [ ]:
import pandas as pd


# Your data: list of lists with two values each.
data1 = OOD_intervention_in_ID

# Create a DataFrame with appropriate column names
df1 = pd.DataFrame(data1, columns=["first", "second"])

# DataFrame 1: Rows where both 'first' and 'second' are nonzero
df_both_nonzero1 = df1[(df1["first"] == 0) & (df1["second"] == 0)].reset_index(drop=True)

# DataFrame 2: Rows where 'first' is nonzero and 'second' is zero
df_first_nonzero_second_zero1 = df1[(df1["first"] != 0) & (df1["second"] == 0)].reset_index(drop=True)

# Display the results
print("DataFrame with both nonzero values:")
print(df_both_nonzero1)

print("\nDataFrame with first nonzero and second zero:")
print(df_first_nonzero_second_zero1)

DataFrame with both nonzero values:
   first  second
0      0       0
1      0       0
2      0       0
3      0       0
4      0       0
5      0       0

DataFrame with first nonzero and second zero:
   first  second
0      1       0
1     15       0
2      1       0
3      1       0
4     16       0
5     24       0
6      2       0
7      1       0
8      1       0
9     18       0


# PURE B fro layer 0 to lyer 4

In [ ]:


res_dict_OOD_intervention_PURE__B= dict()
def replace_hidden_state_B(hidden_to_insert,b, all_hidden_states_to_change,t):
    list_intervention_results = []
    target_layer_intervention = 4
    target_layer_final = 8
    target_B=0
    # juest rename for more standardized structure
    all_hidden_states_ctft_1 = hidden_to_insert

    # we are analysing  with replacing only specific hidden state once a time
    for layer_to_intervene in range(target_layer_intervention , target_layer_intervention +1):
        print(layer_to_intervene)
        hidden_states_1 = all_hidden_states_to_change[layer_to_intervene].clone()    # cloniamo gli hidden del normal run
        hidden_states_ctft_1 = all_hidden_states_ctft_1[target_B]  # copiamo quelli del run che inseriremo
    #r_2  r
        #r_2_ood = all_hidden_states_1_OOD_test[layer_to_intervene]

        # intervene
        #hidden_states_1[0, 0, :] = hidden_states_ctft_1[0, 0, :]   #insereiamo gli hidden state che vogliamo
        hidden_states_1[0, 1, :] = hidden_states_ctft_1[0, 0, :]
        hidden_states_1[0, 2, :] = hidden_states_ctft_1[0, 1, :]  # prova ad inserire r_2 di ood
        #hidden_states_1[0, 2, :]= r_11_id
        #hidden_states_1[0, 2, :]= r_2_ood[0, 2, :] # this is a r(second hop) coming from another ood (with not even a;r_1 the same. This prove that r_2 does not 
        # store calculation in the first 5 layers according to paper results.)
        

        rank_middle_1 = return_rank_norm_layer(hidden_states_1[0, :, :], word_embedding, tokenizer("<"+b+">")['input_ids'][0])[-2]
        res_dict_OOD_intervention["insertion at layer"+str(layer_to_intervene), "B"] = rank_middle_1
        #                
        with torch.no_grad():
            for i in range(layer_to_intervene, target_layer_final):
                f_layer_1 = model.transformer.h[i]  # current layer
                print("layer aggiornato:",i)
                # attention mechanism 
                residual_1 = hidden_states_1       #store the hidden state in residual
                # Apply layer normalization to hidden_states.
                hidden_states_1 = f_layer_1.ln_1(hidden_states_1)
                attn_output_1 = f_layer_1.attn(hidden_states_1)[0] 
                hidden_states_1 = attn_output_1 + residual_1
                # mlp  = Feed-Forward Network:
                residual_1 = hidden_states_1
                hidden_states_1 = f_layer_1.ln_2(hidden_states_1)
                feed_forward_hidden_states_1 = f_layer_1.mlp.c_proj(f_layer_1.mlp.act(f_layer_1.mlp.c_fc(hidden_states_1)))
                hidden_states_1 = residual_1 + feed_forward_hidden_states_1
            # final ln
            hidden_states_1 = model.transformer.ln_f(hidden_states_1)
        # print("--------")
        rank_after_1 = return_rank_norm_layer(hidden_states_1[0, :, :], word_embedding, tokenizer("<"+t+">")['input_ids'][0])[-1]
        res_dict_OOD_intervention["Out-tput layer"+str(target_layer_final), "t"] = rank_after_1
        list_intervention_results.append(rank_middle_1)
        list_intervention_results.append(rank_after_1)
        print("intervention results",res_dict_OOD_intervention)
    print( list_intervention_results)
    return list_intervention_results


In [ ]:
query_change_OOD ="<e_147><r_4><r_1>" #"<e_181><r_6><r_5>"# "<e_147><r_4><r_1>"# "<e_14><r_12><r_13>"       # 2 hop ['<e_11><r_1><e_141></a>'] 

def run_query_B(query_change_OOD, ood=False):
    # inser here the 2-hop query (like ood) from whihc extract the first hop-- "b" activations
    # usato "<e_147><r_4><r_1>"
    h_1_ood,n_r_1_ood,r_1_ood = query_change_OOD.strip("><").split("><") #"e_140","r_19","r_11"             #query.strip("><").split("><")
    b_1_ood = atomic_dict[(h_1_ood ,n_r_1_ood)]
    t_1_ood = atomic_dict[(b_1_ood, r_1_ood)]
    list_b_t=[]

    print("b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa ",b_1_ood)
    print("tail:",t_1_ood)
    
    query_B_layer_for_layer_0=   "<"+ b_1_ood +">"+"<"+ r_1_ood +">"  # here put B and r-2 to extract it at layer 0 and insert it in the norml run

    # Tokenizing the Query:
    decoder_temp_1_OOD = tokenizer([query_B_layer_for_layer_0], return_tensors="pt", padding=True)
    decoder_input_ids_1_OOD, decoder_attention_mask_1_OOD = decoder_temp_1_OOD["input_ids"], decoder_temp_1_OOD["attention_mask"]
    decoder_input_ids_1_OOD, decoder_attention_mask_1_OOD = decoder_input_ids_1_OOD.to(device), decoder_attention_mask_1_OOD.to(device)


    with torch.no_grad():
        # here the same model but with different input !! 
        outputs_1_OOD = model(
            input_ids=decoder_input_ids_1_OOD,
            attention_mask=decoder_attention_mask_1_OOD,
            output_hidden_states=True
        )
    # hidden state of the 1_id (which will be inserted in the normal run)
    all_hidden_states_1_OOD = outputs_1_OOD['hidden_states']

    #################### just to quick check everything is right

    res_dict_b_1_OOD = dict()
    target_layer_b = 4
    target_layer_t = 8
    # ("<"+ b +">")  is just for reconstructing ("<b>") from "b"
    rank_before_1_OOD = return_rank_norm_layer(all_hidden_states_1_OOD[target_layer_b][0, :, :], word_embedding, tokenizer("<"+ b_1_ood +">")['input_ids'][0])[-2]
    res_dict_b_1_OOD['rank_before, layer 5 b'] = rank_before_1_OOD

    # laste layer search for t
    rank_before_1_OOD_t = return_rank_norm_layer(all_hidden_states_1_OOD[target_layer_t][0, :, :], word_embedding, tokenizer("<"+ t_1_ood +">")['input_ids'][0])[-1]
    res_dict_b_1_OOD['rank_before, layer 8 search t'] = rank_before_1_OOD_t
    list_b_t.append((rank_before_1_OOD, rank_before_1_OOD_t))
    if ood:
        print("OOD")
        print(" B 1_OOD",res_dict_b_1_OOD)
        print("as expected in the case of OOD the B is corrctly found at layer 5 but not T at layer 8")
    else:
        print("ID")
        print(" B 1_OOD",res_dict_b_1_OOD)
        print("as expected in the case of ID the B is corrctly found at layer 5 and T at layer 8")

    return all_hidden_states_1_OOD , b_1_ood ,t_1_ood ,list_b_t



In [ ]:
atomic_to_chose = chose_B(199)

In [ ]:


# take 1 from 1_id and 1 from inferred to construct the query
ID_ID_list_P = []
OOD_OOD_list_P = []
OOD_intervention_in_ID_P= []
for i in range(0,7):
    _1_id = atomic_to_chose[0][i]  # list with all the
    _1_ood = atomic_to_chose[1][i]
    _1_id_inferred = atomic_to_chose[2][i]
    _1_ood_inferred = atomic_to_chose[3][i]

    text_1 = _1_id[0]
    match = re.match(r'^(<e_\d+><r_\d+>)', text_1)
    if match:
        result_1_id = match.group(1)

    text_1_1 = _1_id[1]
    match_1 = re.match(r'^(<e_\d+><r_\d+>)', text_1_1)
    if match_1:
        result_1_id_1 = match_1.group(1)

    
    text_2= _1_id_inferred[0]
    match = re.match(r'^<e_\d+>(<r_\d+>)', text_2)
    if match:
        result2_id = match.group(1)

    text_3 = _1_ood[0]
    match = re.match(r'^(<e_\d+><r_\d+>)', text_3)
    if match:
        result_1_ood = match.group(1)
    text_4= _1_ood_inferred[0]
    match = re.match(r'^<e_\d+>(<r_\d+>)', text_4)
    if match:
        result2_ood = match.group(1)

    id_id = result_1_id + result2_id

    id_id_1 = result_1_id_1 + result2_id
    ood_ood = result_1_ood + result2_ood
    print("ID_ID---------------------------------------------------------------------------", id_id)   
    print("OOD_OOD------------------------------------------------------------------------------------", ood_ood)
    normal_run_ID_ID =run_query(id_id, ood=False)
    #B_OOD_insert_activations =run_query(ood_ood, ood=True) 
    normal_2 =run_query_B(id_id , ood=True)  # in this run the insertion is (h,r_1,b) instead of the normla (h,r_1;r_2), to alloe the b and r_1 extraction at layer 0
    #print("-------------------------------------------------------------------------------------------------------------------b",B_OOD_insert_activations[1])
    
    intervention=  replace_hidden_state_B(normal_2[0],normal_2[1],normal_run_ID_ID[0], normal_run_ID_ID[2])  # hidden to insert, b, (all hidden states) normal run, t
    #intervention=  replace_hidden_state_B(normal_2[0],normal_2[1],normal_run_ID_ID[0], normal_run_ID_ID[2])  # hidden to insert, b, (all hidden states) normal run, t
    
    ID_ID_list_P.append(normal_run_ID_ID[3])
    #OOD_OOD_list_P.append(normal_2[3]) # in this case we do not have the second hop insert, and also the inout is different (b,r)instead od (h,r,r)so is normal that we do not find b in fort layer as is insert directlu in layer 0
    OOD_intervention_in_ID_P.append(intervention)  



In [ ]:
ID_ID_list_P

It could be see that by replacing the "pure B"= B from layer 0 to the 4th layer (replacing both B and r_2 from layer 0), the b prediction is correct at layer 4 but the final tail, could be not found as the upper layer never see the atomic facts from the input perspective but always after going through the first 4 layer, leading them to not being able to treat the atomicsfrom layer 0. Similalry, the OOD facts ae never seen from the upper layers , this 1) explains the delayed geenralization in recurrent mode 2) shows that the upper layer never see the OOD atomics at all, as at layer 4 they are so changed to actually see B instead of r_1 . so instead of seeing (B,r_1) they see (B,C) = two consecutive entities , leading the model not beeing able to generlaize ood at all.

In [ ]:
OOD_intervention_in_ID_P